- onednn_verbose format
```
{
  'engine': 'cpu'   
  'prim_kind': 'convolution'   
  'impl': 'brgconv:avx512_core'       
  'prop_kind': 'forward_training'   
  'mds': [
    {'arg': 'src'   'data_type': 'f32'   'properties': ''   'format_kind': 'blocked'   'tag': 'acdb'   'strides': ''   'flags': {'value': 'f0'}}   
    {'arg': 'wei'   'data_type': 'f32'            'properties': ''    'format_kind': 'blocked'   'tag': 'Acdb32a'   'strides': ''   'flags': {'value': 'f0'}}    
    {'arg': 'bia'                         'data_type': 'f32'   'properties': ''          'format_kind': 'blocked'   'tag': 'a'   'strides': ''   'flags': {'value': 'f0'}}   
    {'arg': 'dst'   'data_type': 'f32'   'properties': ''   'format_kind': 'blocked'   'tag': 'acdb'   'strides': ''   'flags': {'value': 'f0'}}
  ]   
  'exts': {
    'attr-post-ops': 
      [{'alg': 'eltwise_relu'   'alpha': '1.0'     'beta': '0.0'      'scale': '1.0'}]
      'attr-scratchpad': 'user'
  }   
  'aux': {'alg': 'convolution_direct'}   
  'shapes': 'mb256_ic256oc512_ih14oh7kh3sh2dh0ph1_iw14ow7kw3sw2dw0pw1'      
  'time': '11.6699\n'
}
```  

In [1]:
!ONEDNN_VERBOSE=1 python3 -W ignore chap05.py --data=/home/taosy/datasets/resnet50 --evaluate --pretrained --profile > /tmp/onednn-1.log  2>/dev/null
!tail -n 20 /tmp/onednn-1.log
!echo =============================================================
!echo
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/onednn-1.log -g breakdown -k prim_kind  impl | column -t -s,

Iteration: 19, inference time: 1.2717132568359375 sec.
 *   Acc@1 76.660 Acc@5 92.676
inference latency: 4.243881 ms
inference Throughput: 235.633353 images/s
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                               aten::mkldnn_convolution        53.07%        8.649s        53.15%        8.661s      21.653ms           400  
                                aten::native_batch_norm        18.77%        3.060s        18.87%        3.074s       7.686ms           400  
enumerate(DataLoader)#_MultiProcessingDataLoaderIter...        12.27%        2.000s        12.27%        2.000s      95.255ms      

In [2]:
!ONEDNN_VERBOSE=1 python3 -W ignore chap05.py --data=/home/taosy/datasets/resnet50 --evaluate --pretrained --ipex --profile > /tmp/onednn-2.log  2>/dev/null
!tail -n 20 /tmp/onednn-2.log
!echo =============================================================
!echo
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/onednn-2.log -g breakdown -k prim_kind  impl | column -t -s,

Iteration: 19, inference time: 0.7899577617645264 sec.
 *   Acc@1 76.660 Acc@5 92.676
inference latency: 3.065839 ms
inference Throughput: 326.175024 images/s
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   torch_ipex::convolution_forward_impl        64.74%        8.645s        66.61%        8.896s      22.240ms           400  
enumerate(DataLoader)#_MultiProcessingDataLoaderIter...        14.40%        1.923s        14.41%        1.924s      91.612ms            21  
                                              aten::add         6.54%     872.948ms         6.54%     872.960ms       5.389ms      

In [3]:
!ONEDNN_VERBOSE=1 python3 -W ignore chap05.py --data=/home/taosy/datasets/resnet50 --evaluate --pretrained --jit --profile > /tmp/onednn-3.log  2>/dev/null
!tail -n 20 /tmp/onednn-3.log
!echo =============================================================
!echo
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/onednn-3.log -g breakdown -k prim_kind  impl | column -t -s,

Iteration: 19, inference time: 0.5521194934844971 sec.
 *   Acc@1 76.660 Acc@5 92.676
inference latency: 2.175135 ms
inference Throughput: 459.741531 images/s
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     ipex_prepack::convolution_relu_run        32.41%        4.361s        34.01%        4.575s      26.755ms           171  
                 ipex_prepack::convolution_add_relu_run        23.36%        3.143s        23.36%        3.143s      20.679ms           152  
                                                forward        17.22%        2.317s        83.62%       11.250s     562.500ms      